# Pseudotemporal Analysis

Looking at the data made avaiable by the [Arlotta lab](https://www.nature.com/articles/s41586-021-03670-5). Please look at the other notebooks and scripts that processed the data. This notebook will analyze the dataset after running scVelo using its [dynamical modeling for pseudotime](https://scvelo.readthedocs.io/en/stable/DynamicalModeling/).

### Imports, globals, etc.

In [1]:
import anndata
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sbn
import scipy
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

In [2]:
plotting_colors = {'Apical progenitors': 'navy',
 'Intermediate progenitors':'steelblue',
 'Red blood cells': 'saddlebrown',
 'VLMC': 'gold',
 'Immature neurons': 'rebeccapurple',
 'Cajal Retzius cells': 'orangered',
 'Microglia': 'pink',
 'Migrating neurons': 'lavender',
 'Endothelial cells': 'lemonchiffon',
 'Pericytes': 'chocolate',
 'Interneurons': 'blue',
 'SCPN': 'green',
 'CThPN': 'darkgreen',
 'DL CPN': 'maroon',
 'UL CPN': 'firebrick',
 'Layer 4': 'darksalmon',
 'Oligodendrocytes': 'mediumvioletred',
 'Layer 6b': 'yellowgreen',
 'NP': 'limegreen',
 'Cycling glial cells': 'lavenderblush',
 'Astrocytes': 'darkmagenta',
 'Ependymocytes': 'blueviolet'}

### Importing h5ad objects

In [3]:
path_to_scVelo = "/home/gridsan/ssauceda/neuroTF_shared/mouse_cortex_development/scVelo/"
file_imputed = "arlotta.imputed.high_var_only.regress_out.umap.filter_splice.scvelo.h5ad"
file_norm = "arlotta.norm.high_var_only.regress_out.umap.filter_splice.scvelo.h5ad"

norm_counts = anndata.read_h5ad(path_to_scVelo + file_norm)
imputed_counts = anndata.read_h5ad(path_to_scVelo + file_imputed)

## Plotting streamplots

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

scv.pl.velocity_embedding_stream(
    adata = norm_counts, 
    color = "New_cellType",
    palette = plotting_colors,
    basis='umap',
    title = "Trajectory of Normalized Counts",
    legend_fontsize = 8,
    arrow_size = 0.7,
    size = 6,
    alpha = 0.3,
)

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

scv.pl.velocity_embedding_stream(
    adata = imputed_counts, 
    color = "New_cellType",
    palette = plotting_colors,
    basis='umap',
    title = "Trajectory of MAGIC Imputed Counts",
    legend_fontsize = 8,
    arrow_size = 0.7,
    size = 4,
    alpha = 0.4,
)

### Note:
- One thing we are considering doing is to filter out interneuron cells for the analysis as they do not play a major role in corticogenesis.

## Plotting velocity pseudotime

In [ ]:
scv.pl.scatter(
    adata = norm_counts, 
    basis = 'umap',
    color='velocity_pseudotime', 
    cmap='gnuplot',
    size = 6,
    alpha = 1,
    title = "Velocity Pseudotime of Normalized Counts"
)

In [ ]:
scv.pl.scatter(
    adata = imputed_counts, 
    basis = 'umap',
    color='velocity_pseudotime', 
    cmap='gnuplot',
    size = 6,
    alpha = 1,
    title = "Velocity Pseudotime of Imputed Counts"
)

## Plotting latent time

In [ ]:
scv.pl.scatter(
    adata = norm_counts, 
    basis = 'umap',
    color='latent_time', 
    cmap='gnuplot',
    size = 6,
    alpha = 1,
    title = "Latent Time of Normalized Counts"
)

In [ ]:
scv.pl.scatter(
    adata = imputed_counts, 
    basis = 'umap',
    color='latent_time', 
    cmap='gnuplot',
    size = 6,
    alpha = 1,
    title = "Latent Time of Imputed Counts"
)

## Plotting PAGA

In [61]:
scv.tl.paga(norm_counts, groups="New_cellType", threshold_root_end_prior=1)

running PAGA using priors: ['velocity_pseudotime']
    finished (0:00:21) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [63]:
df = scv.get_df(norm_counts, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

,Apical progenitors,Astrocytes,CThPN,Cajal Retzius cells,Cycling glial cells,DL CPN,Endothelial cells,Ependymocytes,Immature neurons,Intermediate progenitors,Interneurons,Layer 4,Layer 6b,Microglia,Migrating neurons,NP,Oligodendrocytes,Pericytes,Red blood cells,SCPN,UL CPN,VLMC
Apical progenitors,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Astrocytes,0,0,0,0,0.077,0,0,0,0,0,0,0,0,0,0,0,0.047,0,0,0,0,0
CThPN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cajal Retzius cells,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cycling glial cells,0,0,0,0,0,0,0,0,0,0,0.075,0,0,0,0,0,0,0,0,0,0,0
DL CPN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Endothelial cells,0,0,0,0,0,0,0,0,0,0,0,0.012,0,0,0,0,0,0.5,0,0,0,0
Ependymocytes,0.01,0.031,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Immature neurons,0,0,0,0.046,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.46,0,0
Intermediate progenitors,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.38,0,0,0,0,0,0,0


In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

scv.pl.paga(
    adata = norm_counts, 
    basis='umap', 
    size=50, 
    alpha=.1,
    min_edge_width=2, 
    node_size_scale=1.5
)

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

scv.pl.paga(
    adata = imputed_counts, 
    basis ='umap', 
    size = 50, 
    alpha =.1,
    min_edge_width = 2, 
    node_size_scale = 1.5
)

## Plot gene expression along trajectory

In [ ]:
top_genes = norm_counts.var['fit_likelihood'].sort_values(ascending=False).index[:4]
scv.pl.scatter(
    adata = norm_counts, 
    color = 'New_cellType',
    x = 'velocity_pseudotime', 
    y = top_genes, 
    frameon = False
)

In [ ]:
top_genes = imputed_counts.var['fit_likelihood'].sort_values(ascending=False).index[:4]
scv.pl.scatter(
    adata = imputed_counts, 
    color = 'New_cellType',
    x = 'velocity_pseudotime', 
    y = top_genes, 
    frameon = False
)

## Trajectories

In [ ]:
x, y = scv.utils.get_cell_transitions(adata=norm_counts, basis='umap', starting_cell=70)
ax = scv.pl.velocity_graph(adata=norm_counts, c='lightgrey', edge_width=.05, show=False)
ax = scv.pl.scatter(adata=norm_counts, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)